In [ ]:
!pip install cvlib

In [ ]:
import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox
import os

In [31]:
def detect_and_draw_bbox(img_path, confidence=0.5, model='yolov3-tiny'):
  img = cv2.imread(img_path)
  bbox, label, conf = cv.detect_common_objects(img, confidence=0.5, model='yolov3-tiny')
  bbox_img = draw_bbox(img, bbox, label, conf)

  plt.title(label)
  plt.imshow(bbox_img)
  plt.show()

In [ ]:
!pip install -q fastapi uvicorn pyngrok python-multipart

In [1]:
import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox
import os

from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import StreamingResponse
import uvicorn
import nest_asyncio
from pyngrok import ngrok
from enum import Enum
from io import BytesIO

import numpy as np

In [2]:
class Model(str, Enum):
  yolov3 = 'yolov3'
  yolov3_tiny = 'yolov3-tiny'

In [25]:
app = FastAPI(title="Simple fastapi model")

@app.post('/')
def index(model: Model, img_file: UploadFile=File(...)):
  filename = img_file.filename
  file_ext = filename.split('.')[-1] in ('jpg', 'jpeg', 'png')
  if not file_ext:
    raise HTTPException(status_code=415, detail="Unsupported file provided.")

  img_stream = BytesIO(img_file.file.read())
  img_stream.seek(0)
  file_bytes = np.asarray(bytearray(img_stream.read()), dtype=np.uint8)

  img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
  bbox, label, conf = cv.detect_common_objects(img, model=model)

  out_img = draw_bbox(img, bbox, label, conf)
  cv2.imwrite(filename, out_img)

  file_img = open(filename, mode='rb')

  return StreamingResponse(file_img, media_type='image/jpeg')

In [ ]:
ngrok_tunnel = ngrok.connect(9000)
print(f"Public url: {ngrok_tunnel.public_url}/docs")
nest_asyncio.apply()
uvicorn.run(app, port=9000)